In [12]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pandas as pd
import re 
import string
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords  # Para stopwords (é necessário ter a biblioteca NLTK instalada)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import json

link = "http://localhost:8000"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"}
requisicao = requests.get(link, headers=headers)
site = BeautifulSoup(requisicao.text, "html.parser")
sobre = site.findAll("pre")
profissao = site.findAll("h4", {"class": "profissao"})
codigo = site.findAll("h3", {"class": "codigo"})

sobre_text = [element.get_text() for element in sobre]
profissao_text = [element.get_text() for element in profissao]
codigo_text = [element.get_text() for element in codigo]

dados = {
    'Sobre': sobre_text,
    'Profissao': profissao_text,
    'Codigo': codigo_text
}

df = pd.DataFrame(dados)

df['Profissao'] = df['Profissao'].apply(lambda x: x.lower()) #Conversão para letras minúsculas

# DATA MINING PARA LIMPEZA DE TEXTO:
df['Sobre'] = df['Sobre'].apply(lambda x: x.lower()) #Conversão para letras minúsculas
df['Sobre'] = df['Sobre'].apply(lambda x: re.sub(r'[.,"\'-?:!;]', '', x))
df['Sobre'] = df['Sobre'].apply(lambda x: re.sub('\n', '', x))
df['Sobre'] = df['Sobre'].apply(lambda x: re.sub('\r', '', x))
df['Sobre'] = df['Sobre'].apply(lambda x: re.sub('\t', '', x))
df['Sobre'] = df['Sobre'].apply(lambda x: re.sub('•', '', x))

# TOKEZIZAÇÃO DO TEXTO
# df['Sobre'] = df['Sobre'].apply(lambda x: word_tokenize(x)) #realizar a tokenização dos textos

# VETORIZAÇÃO DO TEXTO, REPRESENTADO OS TEXTOS EM CARACTERES NUMÉRICOS
# USANDO O TF-IDF
# tfidf_vectorizer = TfidfVectorizer() #Cria um objeto TfidfVectorizer
# tfidf_matrix = tfidf_vectorizer.fit_transform(df['Sobre'].apply(lambda x: ' '.join(x))) #Ajusta e transforma os tokens em uma matriz TF-IDF
# df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out()) #Crie um novo DataFrame com as representações TF-IDF

num_clusters = 5

kmeans = KMeans(n_clusters=num_clusters)

def pontuar_curriculo(a, texto):
    conhecimentos = sum(1 for palavra in a["conhecimentos"] if palavra.lower() in texto)
    habilidades = sum(1 for palavra in a["habilidades"] if palavra.lower() in texto)
    atitudes = sum(1 for palavra in a["atitudes"] if palavra.lower() in texto)
    cont_co = len(a["conhecimentos"])
    cont_ha = len(a["habilidades"])
    cont_at = len(a["atitudes"])
    total = conhecimentos + habilidades + atitudes
    porcentagem = (total / (cont_co + cont_ha + cont_at)) * 100
    return conhecimentos, habilidades, atitudes, total, round(porcentagem, 2)

df_pontuado = df.copy()


def match_profissao(a):
    df_pontuado[['Pont_Conhecimentos', 'Pont_Habilidades', 'Pont_Atitudes', 'Total_Pontuacao', 'Porcentagem']] = df_pontuado['Sobre'].apply(lambda x: pd.Series(pontuar_curriculo(a, x)))
    df_p_s = df_pontuado.loc[df['Profissao'] == a["cargo"].lower()]
    kmeans.fit(df_p_s[['Total_Pontuacao']])
    df_p_s['Cluster'] = kmeans.labels_
    profissoes_estruturado = df_p_s[['Codigo','Porcentagem']]
    profissoes_estruturado_decrescente = profissoes_estruturado.sort_values(by='Porcentagem', ascending=False)
    primeiros = profissoes_estruturado_decrescente.head(10)
    profissoes_lista = primeiros.values.flatten().tolist()
    return jsonify([profissoes_lista[i:i+2] for i in range(0, len(profissoes_lista), 2)])

def match_filtroMin(a):
    df_pontuado[['Pont_Conhecimentos', 'Pont_Habilidades', 'Pont_Atitudes', 'Total_Pontuacao', 'Porcentagem']] = df_pontuado['Sobre'].apply(lambda x: pd.Series(pontuar_curriculo(a, x)))
    df_p_s = df_pontuado.loc[df['Profissao'] == a["cargo"].lower()]
    kmeans.fit(df_p_s[['Total_Pontuacao']])
    df_p_s['Cluster'] = kmeans.labels_
    profissoes_estruturado = df_p_s[['Codigo','Porcentagem']]
    profissoes_estruturado_decrescente = profissoes_estruturado.sort_values(by='Porcentagem', ascending=False)
    profissoes_filtro = profissoes_estruturado_decrescente.loc[profissoes_estruturado_decrescente['Porcentagem'] > a['filtro']]
    primeiros = profissoes_filtro.head(10)
    profissoes_lista = primeiros.values.flatten().tolist()
    return jsonify([profissoes_lista[i:i+2] for i in range(0, len(profissoes_lista), 2)])

def match_filtroMax(a):
    df_pontuado[['Pont_Conhecimentos', 'Pont_Habilidades', 'Pont_Atitudes', 'Total_Pontuacao', 'Porcentagem']] = df_pontuado['Sobre'].apply(lambda x: pd.Series(pontuar_curriculo(a, x)))
    df_p_s = df_pontuado.loc[df['Profissao'] == a["cargo"].lower()]
    kmeans.fit(df_p_s[['Total_Pontuacao']])
    df_p_s['Cluster'] = kmeans.labels_
    profissoes_estruturado = df_p_s[['Codigo','Porcentagem']]
    profissoes_estruturado_decrescente = profissoes_estruturado.sort_values(by='Porcentagem', ascending=False)
    profissoes_filtro = profissoes_estruturado_decrescente.loc[profissoes_estruturado_decrescente['Porcentagem'] < a['filtro']]
    primeiros = profissoes_filtro.head(10)
    profissoes_lista = primeiros.values.flatten().tolist()
    return jsonify([profissoes_lista[i:i+2] for i in range(0, len(profissoes_lista), 2)])



app = Flask(__name__)
CORS(app)

@app.route('/scraping', methods=['POST'])
def ask_question():
    try:
        data = request.json
        return match_profissao(data)

    except Exception as e:
        return jsonify({"error": str(e)})

@app.route('/scraping/filtroMin', methods=['POST'])
def filtroMin():
    try:
        data = request.json
        return match_filtroMin(data)

    except Exception as e:
        return jsonify({"error": str(e)})
    
@app.route('/scraping/filtroMax', methods=['POST'])
def filtroMax():
    try:
        data = request.json
        return match_filtroMax(data)

    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=7000)

 * Serving Flask app '__main__'
 * Debug mode: off


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:7000
 * Running on http://192.168.0.10:7000
Press CTRL+C to quit
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\User\AppData\Local\Temp\ipykernel_8204\2000925462.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: